# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Bischöfe einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: `bischof.tbl_bischoefe_gatz` und `bischof.tbl_bischoefe_gatzaemter`

[Bischöfe](#Bischöfe)
- [Einträge in `item`](#Einträge-in-item)
- [Daten eintragen](#Daten-eintragen)
- [Orden eintragen](#Orden-eintragen)
- [Referenzwerke eintragen](#Referenzwerke-eintragen)
- [Externe URLs eintragen](#Externe-URLs-eintragen)
- [Namensvarianten eintragen](#Namensvarianten-eintragen)
- [Suchvarianten eintragen (name_lookup)](#Suchvarianten-eintragen-(name_lookup))

Vorspann: Pakete laden; mit der Datenbank verbinden

In [2]:
wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

In [3]:
cd(wds_path)

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

  Activating project at `C:\Users\Georg\Documents\projekte\WiagDataSetup.jl`


In [6]:
using Revise

In [7]:
using WiagDataSetup

In [8]:
using MySQL, DataFrames

In [9]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

## Bischöfe

In [19]:
df_episc = DBInterface.execute(Wds.dbwiag, "SELECT * FROM bischof.tbl_bischoefe_gatz") |> DataFrame;

Leerzeichen bereinigen

In [20]:
Wds.clean_up!(df_episc)

In [17]:
first(df_episc, 5)

,ID_Bischof,Eintrag RI OPAC,Autoren Gatz,Seite Gatz
,Int32?,String?,String?,String?
1,21086,"um 1372, Johannes",missing,137
2,21087,"um 1378, Michael, Ep. Sythiensis",missing,137
3,21088,"1392-1398, Fr. Dietricus, OFM, Ep. Siginensis",missing,137
4,21089,"1406-1408, Bertholdus, OP, Ep. Sewastopolensis",missing,137
5,21090,"1411-1418, Conradus Simplicy, OFM, Ep. Siginensis",missing,137


In [12]:
names(df_episc)

30-element Vector{String}:
 "ID_Bischof"
 "Eintrag RI OPAC"
 "Autoren Gatz"
 "Seite Gatz"
 "Praefix"
 "Vorname"
 "Familienname"
 "Sterbedatum"
 "Geburtsdatum"
 "Orden"
 "URL_Wikipedia"
 "Weitere Angaben"
 "fertig"
 ⋮
 "Wikidata_ID"
 "GSN_ID"
 "Kommentar_Name"
 "AkademischerTitel"
 "Kommentar_Person"
 "Quelle"
 "QRang_GND"
 "Qualitaet_GND"
 "QRang_GS"
 "Qualitaet_GS"
 "id_bischof_produktiv"
 "id_bischof_lang"

### Einträge in `item`

entsprechende Einträge in Tabelle `item` einfügen

`id` für 'Bischof' in `item_type`: 4

In [51]:
df_item = select(df_episc, :ID_Bischof => :id_in_source, :fertig, Symbol("Eintrag RI OPAC") => :item_in_source);

In [52]:
df_item[!, :item_type_id] .= 4;

In [53]:
map_status(x) = getproperty(x, :bits) > 0 ? "fertig" : missing

map_status (generic function with 1 method)

In [54]:
df_item[!, :edit_status] .= map_status.(df_item[!, :fertig]);

In [69]:
map_online(x) = getproperty(x, :bits) > 0 ? 1 : 0

map_online (generic function with 1 method)

In [70]:
df_item[!, :is_online] .= map_online.(df_item[!, :fertig]);

In [57]:
df_item[!, :created_by] .= 1;

In [58]:
df_item[!, :date_created] .= "2021-10-14 08:40";

In [59]:
df_item[!, :changed_by] .= 1;

In [60]:
df_item[!, :date_changed] .= "2021-10-14 08:40";

In [61]:
function make_id_public(id::Real, num_id_length::Int, id_public_key::AbstractString)
    return "WIAG-" * id_public_key * "-" * lpad(id, num_id_length, '0') * "-001"
end

make_id_public (generic function with 1 method)

In [62]:
make_id_public(3650, 5, "Pers-EPISCGatz")

"WIAG-Pers-EPISCGatz-03650-001"

Normdatum/öffentliche ID erzeugen

Verwende die ursprüngliche ID. Damit entspricht die ID der bisher veröffentlichten ID (2021-10-13)

In [64]:
df_item[!, :id_public] .= ((id) -> make_id_public(id, 5, "Pers-EPISCGatz")).(df_item[!, :id_in_source]);

`item.is_deleted` ist vorbelegt mit `false`

`item.lang` ist vorbelegt mit "de"

In [31]:
names(df_item)

11-element Vector{String}:
 "source_id"
 "fertig"
 "item_in_source"
 "item_type_id"
 "edit_status"
 "is_online"
 "created_by"
 "date_created"
 "changed_by"
 "date_changed"
 "id_public"

In [71]:
df_item[5098:5105, [:id_in_source, :item_type_id, :id_public, :edit_status, :is_online, :date_changed, :changed_by]]

,id_in_source,item_type_id,id_public,edit_status,is_online
,Int32,Int64,String,String?,Int64
1,21699,4,WIAG-Pers-EPISCGatz-21699-001,fertig,1
2,21700,4,WIAG-Pers-EPISCGatz-21700-001,fertig,1
3,21701,4,WIAG-Pers-EPISCGatz-21701-001,fertig,1
4,21702,4,WIAG-Pers-EPISCGatz-21702-001,fertig,1
5,21703,4,WIAG-Pers-EPISCGatz-21703-001,fertig,1
6,21704,4,WIAG-Pers-EPISCGatz-21704-001,fertig,1
7,21705,4,WIAG-Pers-EPISCGatz-21705-001,missing,0
8,21706,4,WIAG-Pers-EPISCGatz-21706-001,fertig,1


In [72]:
Wds.filltable!("item", select(df_item, Not(:fertig)))

┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5171

### Daten eintragen

IDs auslesen

In [22]:
item_type_id = 4
sql = "SELECT id, id_in_source FROM ITEM where item_type_id = ($item_type_id)"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

Spalte für die Zusammenführung befüllen (Typ: String!)

In [23]:
df_episc[!, :id_in_source] .= string.(df_episc[!, :ID_Bischof]);

In [24]:
df_episc_db = innerjoin(df_idx, df_episc, on = :id_in_source);

In [85]:
select_cols = [
    :id,
    :Praefix => :prefixname,
    :Vorname => :givenname,
    :Familienname => :familyname,
    :Sterbedatum => :date_death,
    :Geburtsdatum => :date_birth,
    :Bemerkung_Red => :comment,
    :Kommentar_Name => :note_name,
    :Kommentar_Person => :note_person,
]

9-element Vector{Any}:
                    :id
          :Praefix => :prefixname
          :Vorname => :givenname
     :Familienname => :familyname
      :Sterbedatum => :date_death
     :Geburtsdatum => :date_birth
    :Bemerkung_Red => :comment
   :Kommentar_Name => :note_name
 :Kommentar_Person => :note_person

Einträge anhand der ID in der Quelle löschen (für den Fall eines Updates)

In [118]:
item_type_id = 4
idx_string = join(df_idx[!, :id], ",")
# Es wäre eleganter, hier den Mechanismus von SQL-Parametern zu verwenden. Das funktioniert aber nicht wie erwartet.
sql="DELETE FROM person WHERE id IN ($(idx_string))"
stmt = DBInterface.prepare(Wds.dbwiag, sql);

In [115]:
DBInterface.execute(stmt);

Daten eintragen

In [120]:
Wds.filltable!("person", select(df_episc_db, select_cols...); clear_table=true)

┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5171

### Orden eintragen
Die Ordenstabelle ist vorläufig 2021-10-14


In [189]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) > DataFrame

db_exec (generic function with 1 method)

In [145]:
sql="SELECT DISTINCT Orden as abbreviation FROM bischof.tbl_bischoefe_gatz " *
"WHERE Orden IS NOT NULL AND ORDEN NOT LIKE '1%'"
df_order_col = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [225]:
df_order = DataFrame(abbreviation = [
    "OP",
    "OFM",
    "OPraem",
    "OT",
    "OSB",
    "OCist",
    "OESA",
    "OCarm",
    "CanA",
    "SJ",
    "OMel",
    "OSA",
    "OSM",
    "OFMObs",
    "OFMCap",
    "OFMConv",
    "OFMRec",
    "OCart",
    "OSH",
    "ORC",
    "Otheat",
]);

In [244]:
df_order[!, :id_in_source] .= string.(1:size(df_order, 1));

Einträge in `item`

In [245]:
df_item = select(df_order, :id_in_source);

In [246]:
df_item[!, :item_type_id] .= 7;

In [247]:
df_item[!, :is_online] .= 0;

In [248]:
df_item[!, :created_by] .= 7;

In [249]:
df_item[!, :date_created] .= "2021-10-14 13:30";

In [250]:
df_item[!, :changed_by] .= 7;

In [251]:
df_item[!, :date_changed] .= "2021-10-14 13:30";

In [252]:
item_type_id

6

In [253]:
sql = "DELETE FROM item WHERE item_type_id = $(item_type_id)";
db_exec(sql);

In [254]:
Wds.filltable!("item", df_item)

┌ Info: Rows inserted: 21
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


21

Daten eintragen in `religious_order_id`

Index-Werte lesen

In [255]:
item_type_id = 6
sql = "SELECT id, id_in_source FROM ITEM where item_type_id = ($item_type_id)"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [256]:
df_order_db = innerjoin(df_idx, df_order, on = :id_in_source);

In [257]:
Wds.clean_up!(df_order_db)

In [242]:
names(df_order_db)

3-element Vector{String}:
 "id"
 "id_in_source"
 "abbreviation"

In [258]:
Wds.filltable!("religious_order", select(df_order_db, Not(:id_in_source)); clear_table=true)

┌ Info: Rows inserted: 21
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


21

IDs in Bischofstabelle eintragen

*TODO* Ersatzfeld in `item_property` verwenden, falls der Orden nicht eindeutig zugeordnet werden kann?

In [259]:
df_episc_order = dropmissing(df_episc_db, :Orden);

In [260]:
Wds.clean_up!(df_episc_order)

In [261]:
size(df_episc_order)

(776, 32)

In [262]:
df_episc_order[!, :join_field] .= lowercase.(df_episc_order[!, :Orden]);

In [264]:
df_order_db[!, :join_field] .= lowercase.(df_order_db[!, :abbreviation]);

In [266]:
df_episc_order_id = leftjoin(df_episc_order, 
    select(df_order_db, :id => :religious_order_id, :join_field), 
    on = :join_field);

In [272]:
size(df_episc_order_id)

(776, 34)

In [274]:
df_episc_order_id = dropmissing(df_episc_order_id, :religious_order_id);

In [275]:
size(df_episc_order_id)

(756, 34)

In [276]:
df_episc_order_id[300:310, [:id, :id_in_source, :Vorname, :religious_order_id, :Orden]]

,id,id_in_source,Vorname,religious_order_id,Orden
,Int32,SubStri…,SubStri…,Int32,SubStri…
1,1231,3396,Ludwig,5312,OT
2,1230,3397,Heinrich,5310,OFM
3,1229,3398,Olav,5310,OFM
4,1228,3399,Johannes,5312,OT
5,1227,3400,Dietrich,5312,OT
6,1225,3402,Arnold,5312,OT
7,1224,3403,Heinrich,5312,OT
8,1223,3405,Bertholt,5314,OCist
9,1221,3407,Albert,5309,OP


Im ersten Versuch war Groß-/Kleinschreibung unterschieden worden, dadurch gingen einige Einträge verloren.

In [268]:
fcid(id) = id in (1070, 211, 951, 4145, 4282, 4689, 5237)

fcid (generic function with 1 method)

In [269]:
filter(:id => fcid, select(df_episc_order_id, :id, :ID_Bischof, :Vorname, :Familienname, :Orden, :religious_order_id))

,id,ID_Bischof,Vorname,Familienname,Orden,religious_order_id
,Int32,Int32,SubStri…,SubStri…?,SubStri…,Int32?
1,1070,21096,Heinrich,Ringwirt,OFM,5310
2,951,21140,Kuno,missing,Ocist,5314
3,211,2804,Konrad,Krosigk,Ocist,5314
4,4689,20326,Heinrich,Hattingen,Ocarm,5316
5,4282,21268,Hartung,missing,OMel,5319
6,4145,21489,Jakob,Denmarcken,OMel,5319
7,5237,21761,Johannes,Reuther,Ocarm,5316


Tabelle `person` aktualisieren

In [277]:
DBInterface.execute(Wds.dbwiag, "DROP TABLE IF EXISTS person_update")
sql = "CREATE TEMPORARY TABLE person_update (" *
"id INT, " *
"religious_order_id INT)"
DBInterface.execute(Wds.dbwiag, sql);

In [278]:
Wds.filltable!("person_update", select(df_episc_order_id, :id, :religious_order_id))

┌ Info: Rows inserted: 756
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


756

In [279]:
sql = "UPDATE person as p, (SELECT id, religious_order_id FROM person_update) as oid " *
"SET p.religious_order_id = oid.religious_order_id " *
"WHERE p.id = oid.id"
DBInterface.execute(Wds.dbwiag, sql);

### numerische Daten eintragen

In [281]:
parse_date_birth(s) = Wds.parsemaybe(s, :lower)

parse_date_birth (generic function with 1 method)

In [284]:
col_numdate_birth = parse_date_birth.(df_episc_db[!, :Geburtsdatum]);

In [286]:
parse_date_death(s) = Wds.parsemaybe(s, :upper)

parse_date_death (generic function with 1 method)

In [287]:
col_numdate_death = parse_date_death.(df_episc_db[!, :Sterbedatum]);

In [294]:
col_numdate_birth[515:525]

11-element Vector{Union{Missing, Int64}}:
     missing
     missing
     missing
 1315
     missing
 1335
 1355
 1345
     missing
     missing
     missing

In [308]:
df_episc_db[!, :num_date_birth] = col_numdate_birth;

In [309]:
df_episc_db[!, :num_date_death] = col_numdate_death;

In [300]:
df_episc_db[530:535, [:id, :id_in_source, :ID_Bischof, :Vorname, :numdate_birth, :numdate_death]]

,id,id_in_source,ID_Bischof,Vorname,numdate_birth,numdate_death
,Int32,String?,Int32,SubStri…,Int64?,Int64?
1,550,3699,3699,Gerhard,missing,1278
2,549,3700,3700,Robert,missing,1271
3,548,3701,3701,Ulrich,missing,1274
4,547,3702,3702,Jakob,missing,1297
5,546,3703,3703,Heinrich,missing,1286
6,545,3704,3704,Nikolaus,missing,1362


In [313]:
update_table = "person_date_update"
sql = "DROP TABLE IF EXISTS $(update_table)"
db_exec(sql)
sql = "CREATE TEMPORARY TABLE $(update_table) (" *
"id INT, " *
"num_date_birth INT, " *
"num_date_death INT)"
db_exec(sql);

In [314]:
Wds.filltable!(update_table, select(df_episc_db, :id, :num_date_birth, :num_date_death))

┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5171

In [316]:
sql = "UPDATE person as p, (SELECT * FROM $(update_table)) as udt " *
"SET p.num_date_birth = udt.num_date_birth, p.num_date_death = udt.num_date_death " *
"WHERE p.id = udt.id"
db_exec(sql);

### Referenzwerke eintragen

Quelle: Tabelle `wiag.reference`

In [11]:
sql = "SELECT * from wiag.reference WHERE id_ref <= 5";
df_ref_gatz = Wds.sql_df(sql);

In [12]:
item_type_id = 4
df_ref_gatz[!, :item_type_id] .= 4;

In [15]:
columns = [
    :item_type_id => :item_type_id,
    :id_ref => :reference_id,
    :full_citation => :full_citation,
    :author_editor => :author_editor,
    :onlineressource => :online_resource,
    :short_title => :title_short,
    :ri_opac_id => :ri_opac_id,
    :year_of_publication => :year_publication,
    :isbn => :isbn,
    :sort => :gs_display_order,
];

In [18]:
table_name = "reference_volume"
Wds.filltable!(table_name, select(df_ref_gatz, columns), clear_table=true)

┌ Info: Rows inserted: 5
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1204


5

### Verweise für die einzelnen Bischöfe

In [30]:
columns = [
    :id => :item_id,
    Symbol("Seite Gatz") => :page,
    :Quelle => :reference_id
]

3-element Vector{Pair{Symbol, Symbol}}:
                  :id => :item_id
 Symbol("Seite Gatz") => :page
              :Quelle => :reference_id

In [31]:
df_episc_ref = select(df_episc_db, columns);

In [32]:
df_episc_ref[!, :item_type_id] .= item_type_id;

`21576 Ludovico Trevisan` hat keinen Verweis auf eine Quelle

In [34]:
dropmissing!(df_episc_ref, :reference_id);

In [318]:
names(df_episc_ref)

3-element Vector{String}:
 "item_id"
 "page"
 "reference_volume_id"

Einträge löschen im Fall von Korrekturen

In [328]:
idx_str = join(df_episc_ref[!, :item_id], ",")
sql = "DELETE FROM item_reference WHERE item_id in ($(idx_string))"
db_exec(sql);

Einträge löschen für einen vollständigen neuen Satz

In [36]:
sql = "DELETE FROM item_reference WHERE item_type_id = $(item_type_id)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM item_reference WHERE item_type_id = 4", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [37]:
Wds.filltable!("item_reference", df_episc_ref, clear_table = false)

┌ Info: Rows inserted: 5170
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1204


5170

### Externe URLs eintragen

In [10]:
using MySQL, DataFrames

In [156]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [10]:
item_type_id = 4

4

In [11]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame

db_exec (generic function with 1 method)

Einträge löschen (für den Fall eines Fehlers oder für ein Update)

In [55]:
sql = "DELETE FROM id_external " *
"WHERE item_id IN (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
db_exec(sql);

In [15]:
sql = "SELECT id, url_name_formatter FROM authority " *
"WHERE id IN (1, 2, 3, 4, 5, 200)"
db_exec(sql)

,id,url_name_formatter
,Int32,String?
1,1,Gemeinsame Normdatei (GND) ID
2,2,Wikidata
3,3,Wikipedia-Artikel
4,4,VIAF-ID
5,5,WIAG-ID
6,200,Personendatenbank der Germania Sacra


Index aus `item` lesen

In [12]:
sql = "SELECT id, id_in_source FROM item WHERE item_type_id = $(item_type_id)"
df_idx = db_exec(sql); 

In [13]:
size(df_idx)

(5171, 2)

Externe Ids aus den Quelldaten lesen  
`bischof.tbl_bischoefe_gatz`

In [14]:
sql = "SELECT ID_Bischof, Vorname, Familienname, " *
"VIAF_ID, GND_ID, Wikidata_ID, GSN_ID, URL_Wikipedia " *
"FROM bischof.tbl_bischoefe_gatz"
df_src = db_exec(sql);

In [17]:
Wds.clean_up!(df_src)

In [18]:
df_src[!, :id_in_source] .= string.(df_src[!, :ID_Bischof]);

**VIAF**, dann GND, Wikidata, GSN, Wikipedia

In [20]:
authority_id_viaf = 4 # siehe oben

4

In [48]:
df_src_viaf = dropmissing(df_src, :VIAF_ID)
df_src_viaf[!, :authority_id] .= authority_id_viaf;
df_ins_0 = innerjoin(df_idx, df_src_viaf, on = :id_in_source);

In [50]:
size(df_ins_0)

(2971, 11)

In [51]:
df_ins = select(df_ins_0, :id => :item_id, :authority_id, :VIAF_ID => :value);

In [52]:
df_ins[300:307, :]

,item_id,authority_id,value
,Int32,Int64,SubStri…
1,585,4,20437728
2,586,4,32910366
3,587,4,120776855
4,588,4,81533451
5,590,4,86138524
6,591,4,2873713
7,592,4,60150995
8,593,4,221043719


In [56]:
Wds.filltable!("id_external", df_ins)

┌ Info: Rows inserted: 2971
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


2971

**GND**

In [20]:
authority_id_gnd = 1 # siehe oben

1

In [24]:
df_src_gnd = dropmissing(df_src, :GND_ID)
df_src_gnd[!, :authority_id] .= authority_id_gnd;
df_ins_0 = innerjoin(df_idx, df_src_gnd, on = :id_in_source);

In [25]:
size(df_ins_0)

(2866, 11)

In [26]:
df_ins = select(df_ins_0, :id => :item_id, :authority_id, :GND_ID => :value);

In [27]:
df_ins[300:307, :]

,item_id,authority_id,value
,Int32,Int64,SubStri…
1,588,1,137327579
2,590,1,137980302
3,591,1,103078711
4,592,1,129021709
5,593,1,189543795
6,594,1,129068225
7,595,1,129305774
8,597,1,1024543595


In [28]:
Wds.filltable!("id_external", df_ins)

┌ Info: Rows inserted: 2866
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


2866

**Wikidata**

In [29]:
authority_id_wd = 2 # siehe oben

2

In [30]:
df_src_wd = dropmissing(df_src, :Wikidata_ID)
df_src_wd[!, :authority_id] .= authority_id_wd;
df_ins_0 = innerjoin(df_idx, df_src_wd, on = :id_in_source);

In [31]:
size(df_ins_0)

(5171, 11)

In [34]:
df_ins = select(df_ins_0, :id => :item_id, :authority_id, :Wikidata_ID => :value);

In [35]:
df_ins[300:307, :]

,item_id,authority_id,value
,Int32,Int64,SubStri…
1,780,2,Q55810243
2,779,2,Q55786070
3,778,2,Q55803565
4,777,2,Q1696813
5,776,2,Q1696339
6,775,2,Q97699216
7,774,2,Q1549799
8,773,2,Q55903980


In [36]:
Wds.filltable!("id_external", df_ins)

┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5171

**GSN**

In [37]:
authority_id_gsn = 200 # siehe oben

200

In [40]:
df_src_gsn = dropmissing(df_src, :GSN_ID)
df_src_gsn[!, :authority_id] .= authority_id_gsn;
df_ins_0 = innerjoin(df_idx, df_src_gsn, on = :id_in_source);

In [41]:
size(df_ins_0)

(2113, 11)

In [42]:
df_ins = select(df_ins_0, :id => :item_id, :authority_id, :GSN_ID => :value);

In [43]:
df_ins[300:307, :]

,item_id,authority_id,value
,Int32,Int64,SubStri…
1,701,200,056-00109-001
2,702,200,056-01525-001
3,703,200,056-01303-001
4,704,200,056-01224-001
5,705,200,056-01111-001
6,706,200,056-00830-001
7,707,200,056-00558-001
8,708,200,056-01697-001


In [44]:
Wds.filltable!("id_external", df_ins)

┌ Info: Rows inserted: 2113
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


2113

**Wikipedia**

In [15]:
authority_id_wp = 3 # siehe oben

3

In [19]:
df_src_wp = dropmissing(df_src, :URL_Wikipedia)
df_src_wp[!, :authority_id] .= authority_id_wp;
df_ins_0 = innerjoin(df_idx, df_src_wp, on = :id_in_source);

In [20]:
size(df_ins_0)

(2938, 11)

In [37]:
df_ins = select(df_ins_0, :id => :item_id, :authority_id, :URL_Wikipedia => :value);

Die Wikipedia-URLs sind teilweise mit '#' versehen und doppelt  
#https://de.wikipedia.org/wiki/Ruprecht_von_der_Pfalz_%28Freising%29#https://de.wikipedia.org/wiki/Ruprecht_von_der_Pfalz_%28Freising%29#

In [22]:
url_m = "#https://de.wikipedia.org/wiki/Ruprecht_von_der_Pfalz_%28Freising%29#https://de.wikipedia.org/wiki/Ruprecht_von_der_Pfalz_%28Freising%29#"
Wds.fix_Wikipedia_URL(url_m)

"Ruprecht_von_der_Pfalz_%28Freising%29"

In [ ]:
df_ins[!, :value] .= Wds.fix_Wikipedia_URL.(df_ins[!, :value]);

In [39]:
df_ins[300:307, :]

,item_id,authority_id,value
,Int32,Int64,SubStri…
1,463,3,Timo_(Bamberg)
2,464,3,Anselm_von_Nenningen
3,465,3,Friedrich_von_Grafeneck
4,467,3,Burkhard_von_Ellerbach
5,468,3,Johann_Schadland
6,469,3,Walter_II._von_Hochschlitz
7,470,3,Marquard_I._von_Randeck
8,474,3,Degenhard_von_Hellenstein


debug begin

In [57]:
using CSV

In [58]:
file_wp = "C:\\Users\\georg\\Documents\\tmp\\bishops_url_wikidata.csv"

"C:\\Users\\georg\\Documents\\tmp\\bishops_url_wikidata.csv"

In [59]:
CSV.write(file_wp, df_ins)

"C:\\Users\\georg\\Documents\\tmp\\bishops_url_wikidata.csv"

debug end

In [40]:
Wds.filltable!("id_external", df_ins)

┌ Info: Rows inserted: 2938
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


2938

### Namensvarianten eintragen

In [7]:
using MySQL, DataFrames

In [8]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame

db_exec (generic function with 1 method)

In [9]:
table_src = "bischof.tbl_bischoefe_gatz"
sql = "SELECT ID_Bischof, Vorname, Familienname, Vorname_Varianten, Familienname_Variante " *
"FROM $(table_src)"
df_src = db_exec(sql);

LoadError: UndefVarError: Wds not defined

In [45]:
Wds.clean_up!(df_src)

In [65]:
df_fnv = dropmissing(df_src, :Familienname_Variante);

In [66]:
size(df_fnv)

(535, 5)

In [67]:
df_fnv[200:207, :]

,ID_Bischof,Vorname,Familienname,Vorname_Varianten,Familienname_Variante
,Int32,SubStri…,SubStri…?,SubStri…?,SubStri…
1,5344,Barthélemy,Chuet,Bartholomaeus,"Chouet, Cueti"
2,5347,Judocus,Dale,missing,de Valle
3,5352,Johannes,Dieburger,missing,Dieperger
4,5357,Hiob,Dobeneck,missing,Job
5,5364,Wilhelm,Enckenvoirt,missing,Enckevort
6,5367,Albert,Engel,missing,"Enghel, Engelen"
7,5374,Johannes,Fabri,missing,Smed
8,5425,Hermann,missing,missing,Weise


In [69]:
df_exp = Wds.expand_column(df_fnv, :Familienname_Variante, delim = ",");

In [70]:
size(df_exp)

(698, 5)

In [71]:
df_exp[150:157, :]

,ID_Bischof,Vorname,Familienname,Vorname_Varianten,Familienname_Variante
,Int32,SubStri…,SubStri…?,SubStri…?,SubStri…
1,3225,Johannes,Krouwel,missing,Crowel
2,3225,Johannes,Krouwel,missing,Creul
3,3232,Bruno,Schauenburg,missing,Schaumburg
4,3234,Johann,der Kahle von Wallenstein,missing,Holý von Wallenstein
5,3237,Hynko,Berka von Duba,Hynek,Berka z Dubé
6,3241,Peter,Wurst,missing,Jelito
7,3243,Latzek,Krawarn,Ladislaus,Kravare
8,3244,Johann,Frost,missing,Mráz


In [72]:
df_exp[!, :id_in_source] .= string.(df_exp[!, :ID_Bischof]);

In [73]:
df_ins = innerjoin(df_idx, df_exp, on = :id_in_source);

In [74]:
size(df_ins)

(698, 7)

Spalte `lang` hat den Default-Wert 'de'.

In [76]:
table_dst = "familyname_variant"
Wds.filltable!(table_dst, select(df_ins, :id => :person_id, :Familienname_Variante => :name))

┌ Info: Rows inserted: 698
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


698

In [46]:
df_gnv = dropmissing(df_src, :Vorname_Varianten);

In [52]:
size(df_gnv)

(304, 5)

In [48]:
df_gnv[200:207, :]

,ID_Bischof,Vorname,Familienname,Vorname_Varianten,Familienname_Variante
,Int32,SubStri…,SubStri…?,SubStri…,SubStri…?
1,4501,Goericus,missing,Abbo,missing
2,4531,Erkanbert,missing,Herkumbert,missing
3,12635,Georg,Lock,Franz Georg,missing
4,4548,Eilbert,missing,Egilbert,missing
5,4551,Ulrich,missing,Odalrich,missing
6,4600,Vercel,missing,Vezel,missing
7,4608,Dietleb,missing,Detlef,missing
8,4645,Dodo,missing,Udo,missing


In [92]:
df_exp = Wds.expand_column(df_gnv, :Vorname_Varianten, delim = r", *");

In [93]:
size(df_exp)

(331, 5)

In [94]:
df_exp[150:157, :]

,ID_Bischof,Vorname,Familienname,Vorname_Varianten,Familienname_Variante
,Int32,SubStri…,SubStri…?,SubStri…,SubStri…?
1,4389,Sophronius,missing,Suffronius,missing
2,4395,Lesio,missing,Leudegasius,missing
3,4396,Becilin,missing,Petelinus,missing
4,4396,Becilin,missing,Bothadus ?,missing
5,4397,Leoboald,missing,Ludoaldus,missing
6,4398,Lantwald,missing,Lanwald,missing
7,4398,Lantwald,missing,Lantivaldus,missing
8,4399,Riuthard,missing,Ruthardus ?,missing


Manche Namenseinträge enthalten Fragezeichen. Diese bleiben hier erhalten, werden aber für die Tabelle `name_lookup` entfernt.

In [58]:
df_exp[!, :id_in_source] .= string.(df_exp[!, :ID_Bischof]);

In [60]:
df_ins = innerjoin(df_idx, df_exp, on = :id_in_source);

In [61]:
size(df_ins)

(331, 7)

Spalte `lang` hat den Default-Wert 'de'.

In [64]:
table_dst = "givenname_variant"
Wds.filltable!(table_dst, select(df_ins, :id => :person_id, :Vorname_Varianten => :name))

┌ Info: Rows inserted: 331
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


331

### Suchvarianten eintragen (name_lookup)

Test

In [88]:
Wds.makevariantsgn("Klaus Walter", "von der", "Vogelweide", "Vollmer, Liber")

6-element Vector{Vector{Union{Missing, String}}}:
 ["Klaus Walter", "von der", "Vogelweide", "Klaus Walter Vogelweide", "Klaus Walter von der Vogelweide"]
 ["Klaus", "von der", "Vogelweide", "Klaus Vogelweide", "Klaus von der Vogelweide"]
 ["Klaus Walter", "von der", "Vollmer", "Klaus Walter Vollmer", "Klaus Walter von der Vollmer"]
 ["Klaus", "von der", "Vollmer", "Klaus Vollmer", "Klaus von der Vollmer"]
 ["Klaus Walter", "von der", "Liber", "Klaus Walter Liber", "Klaus Walter von der Liber"]
 ["Klaus", "von der", "Liber", "Klaus Liber", "Klaus von der Liber"]

In [29]:
df_src = DataFrame(
    id = [1077, 31, 32, 33, 34, 35, 36],
    givenname = ["Dietricus", "Albert", "Hans Otto", "Wilhelm", "Kurt Georg", "Otto", "Otto"],
    prefix_name = [missing, "von", "zu", "von", missing, "auf der", missing],
    familyname = [missing, "Häusler", "Oberhof", "Berg", missing, "Scholle", "Weiß"],
    givenname_variant = [missing, missing, "Johann Otto", "Guido, Guillaume", "Konrad", missing, "Odilon"],
    familyname_variant = [missing, "Hauser, Schad", missing, "Berge", missing, missing, "Blanc"]
    )

,id,givenname,prefix_name,familyname,givenname_variant,familyname_variant
,Int64,String,String?,String?,String?,String?
1,1077,Dietricus,missing,missing,missing,missing
2,31,Albert,von,Häusler,missing,"Hauser, Schad"
3,32,Hans Otto,zu,Oberhof,Johann Otto,missing
4,33,Wilhelm,von,Berg,"Guido, Guillaume",Berge
5,34,Kurt Georg,missing,missing,Konrad,missing
6,35,Otto,auf der,Scholle,missing,missing
7,36,Otto,missing,Weiß,Odilon,Blanc


In [30]:
df_nl = Wds.create_name_lookup(df_src) # schaut gut aus

,person_id,gn_fn,gn_prefix_fn
,Int64,String?,String?
1,1077,Dietricus,missing
2,31,Albert Häusler,Albert von Häusler
3,31,Albert Hauser,Albert von Hauser
4,31,Albert Schad,Albert von Schad
5,32,Hans Otto Oberhof,Hans Otto zu Oberhof
6,32,Hans Oberhof,Hans zu Oberhof
7,32,Johann Otto Oberhof,Johann Otto zu Oberhof
8,32,Johann Oberhof,Johann zu Oberhof
9,33,Wilhelm Berg,Wilhelm von Berg


In [104]:
using MySQL, DataFrames

In [ ]:
db_exec(sql) = DBInterface(Wds.dbwiag, sql)

ID auslesen

In [39]:
item_type_id = 4
sql = "SELECT id, id_in_source FROM ITEM where item_type_id = ($item_type_id)"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [37]:
table_src = "bischof.tbl_bischoefe_gatz"
sql = "SELECT concat(ID_Bischof) AS id_in_source, " *
"Vorname AS givenname, " *
"Praefix AS prefix_name, " *
"Familienname AS familyname," * 
"Vorname_Varianten AS givenname_variant, " * 
"Familienname_Variante AS familyname_variant " *
"FROM $(table_src)"
df_src = db_exec(sql);

In [38]:
Wds.clean_up!(df_src)

In [21]:
size(df_src)

(5171, 6)

In [40]:
names(df_src)

6-element Vector{String}:
 "id_in_source"
 "givenname"
 "prefix_name"
 "familyname"
 "givenname_variant"
 "familyname_variant"

In [41]:
df_nl_in = innerjoin(df_idx, df_src, on = :id_in_source);

In [42]:
size(df_nl_in)

(5171, 7)

In [43]:
df_nl = Wds.create_name_lookup(df_nl_in);

In [44]:
size(df_nl)

(7311, 3)

In [45]:
df_nl_empty = filter(:gn_fn => ismissing, df_nl);

In [47]:
size(df_nl_empty)

(0, 3)

In [48]:
df_nl[2300:2312, :]

,person_id,gn_fn,gn_prefix_fn
,Int64,String?,String?
1,1213,Fromhold Vifhusen,Fromhold von Vifhusen
2,1212,Johannes Walteri von Sinten,missing
3,1211,Siegfried Blomberg,missing
4,1210,Johannes Ambundii,missing
5,1209,Henning Scharpenberg,missing
6,1208,Adalbert Böhmen,Adalbert von Böhmen
7,1207,Eberhard Regensberg,Eberhard von Regensberg
8,1206,Burkhard Ziegenhain,Burkhard von Ziegenhain
9,1205,Ulrich,missing


In [49]:
table_dst = "name_lookup"
Wds.filltable!(table_dst, df_nl)

┌ Info: Rows inserted: 7311
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


7311